In [2]:
import pandas as pd
import numpy as np
import time
from neo4j import GraphDatabase

In [3]:
df = pd.read_csv('Datasets\dataset_final.csv')

In [3]:
uri = "neo4j+s://b073ebf5.databases.neo4j.io"
user = "neo4j"
password = "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE"

driver = GraphDatabase.driver(uri, auth=(user, password))

# Test de conexión
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexión exitosa con AuraDB' AS mensaje")
        for record in result:
            print(record["mensaje"])

test_connection()

Conexión exitosa con AuraDB


In [4]:
def crear_nodo_depto(tx, datos):
    query = """
    CREATE (:Departamento {
        id: $id,
        latitude: $latitude,
        longitude: $longitude
    })
    """
    tx.run(query, **datos)
    
def cargar_deptos(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            datos = {
                "id": row["id"],
                "latitude": float(row["latitud"]),
                "longitude": float(row["longitud"])
            }
            session.write_transaction(crear_nodo_depto, datos)

In [9]:
start = time.time()
cargar_deptos(df)
end = time.time()
print(f'Tiempo de ejecucion = {end-start} segundos')

C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_26040\759094799.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(crear_nodo_depto, datos)
[#D27A]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Unable to retrieve routing information
Transaction failed and will be retried in 1.09967258644022s (Unable to retrieve routing information)


Tiempo de ejecucion = 10157.992450237274 segundos


In [18]:
df_pois = pd.read_csv('Datasets\pois_final.csv')
df_pois["poi_id"] = df_pois.index.astype('int64')
df_pois["name"] = df_pois["name"].fillna("NoName")
df_pois.sample(10)

,longitude,latitude,type,class,name,poi_id
3959,-70.674136,-33.396817,School,education_prim,Escuela Especial Reina María,3959
5400,-70.647873,-33.468365,School,education_prim,SaLa Cuna y Jardín Infantil Institución Sweet,5400
5229,-70.673289,-33.412314,School,education_prim,Escuela Básica Santa Teresita,5229
18762,-70.577722,-33.551605,School,education_prim,Escuela Basica Santa Ana De La Luz,18762
755,-70.661414,-33.489340,Butcher,food_and_drink_stores,Carnes RV,755
5670,-70.673193,-33.440730,Cafe,food_and_drink,Brunet,5670
17254,-70.773322,-33.528985,School,education_prim,Escuela Básica Los Bosquinos,17254
13216,-70.719755,-33.492898,Clinic,medical,Cesfam Dr. Norman Voulliéme,13216
3053,-70.519683,-33.349439,Swimming Pool,sport_and_leisure,Club de Campo Colegio Médico,3053
17033,-70.672546,-33.527230,Fitness,sport_and_leisure,NoName,17033


In [19]:
df_pois.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21444 entries, 0 to 21443
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   longitude  21444 non-null  float64
 1   latitude   21444 non-null  float64
 2   type       21444 non-null  object 
 3   class      21444 non-null  object 
 4   name       21444 non-null  object 
 5   poi_id     21444 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1005.3+ KB


In [22]:
def crear_nodos_pois(tx, row):
    query = """
    MERGE (p:POI {
        poi_id: $poi_id
    })
    SET p.name = $name,
        p.latitude = $latitude,
        p.longitude = $longitude,
        p.class = $class
    """
    tx.run(query, **row)

def cargar_pois(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            datos = {
                "poi_id": row["poi_id"],
                "name": row["name"],
                "latitude": float(row["latitude"]),
                "longitude": float(row["longitude"]),
                "class": row["class"]
            }
            session.write_transaction(crear_nodos_pois, datos)

In [24]:
start = time.time()
cargar_pois(df_pois)
end = time.time()
print(f'Tiempo de ejecucion = {end-start} segundos')

C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_30280\3023171507.py:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(crear_nodos_pois, datos)
[#E2A8]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Unable to retrieve routing information
Transaction failed and will be retried in 0.8504545047050933s (Unable to retrieve routing information)


Tiempo de ejecucion = 8770.718525409698 segundos


In [4]:
import pandas as pd
import numpy as np

In [10]:
df_depto = pd.read_csv('Datasets/dataset_final.csv')

In [5]:
df_poi = pd.read_csv('Datasets/pois_final.csv')
df_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21444 entries, 0 to 21443
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   longitude  21444 non-null  float64
 1   latitude   21444 non-null  float64
 2   type       21444 non-null  object 
 3   class      21444 non-null  object 
 4   name       21444 non-null  object 
 5   cat        21444 non-null  int64  
 6   poi_id     21444 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 1.1+ MB


In [13]:
df_poi['class'].unique()

array(['sport_and_leisure', 'medical', 'education_prim', 'veterinary',
       'food_and_drink_stores', 'arts_and_entertainment',
       'food_and_drink', 'park_like', 'security', 'religion',
       'education_sup'], dtype=object)

In [6]:
df_metro= pd.read_csv('Datasets/metro.csv')
df_metro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           143 non-null    int64  
 1   linea        143 non-null    object 
 2   orden        143 non-null    int64  
 3   combinacion  30 non-null     object 
 4   nombre       143 non-null    object 
 5   longitud     143 non-null    float64
 6   latitud      143 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 7.9+ KB


In [8]:
df_bus=pd.read_csv('Datasets/Paraderos.csv')
df_bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12635 entries, 0 to 12634
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         12635 non-null  int64  
 1   id          12635 non-null  int64  
 2   codigo      12635 non-null  object 
 3   simt        11774 non-null  object 
 4   tipo        12635 non-null  object 
 5   clasificac  12635 non-null  object 
 6   comuna      12635 non-null  object 
 7   zona        12635 non-null  object 
 8   eje         12635 non-null  object 
 9   desde       12432 non-null  object 
 10  hacia       12631 non-null  object 
 11  sentido     12635 non-null  object 
 12  fila_super  12635 non-null  object 
 13  fila_infer  12635 non-null  object 
 14  grupo_de_p  12635 non-null  object 
 15  identifica  12634 non-null  object 
 16  nombre_ust  12635 non-null  object 
 17  x           12635 non-null  float64
 18  y           12635 non-null  float64
 19  zona_paga   12635 non-nul

In [12]:
import folium

# Centro aproximado de Santiago de Chile 
centro_santiago = [-33.45, -70.66]

# Crear el mapa
mapa = folium.Map(location=centro_santiago, zoom_start=12)

# --- Opcional pero útil: capas separadas por tipo ---
depto_layer = folium.FeatureGroup(name="Depto")
pois_layer = folium.FeatureGroup(name="POIs")
bus_layer = folium.FeatureGroup(name="Bus")
metro_layer = folium.FeatureGroup(name="Metro")

# ======================
#  Deptos (df_depto) - blue
# ======================
for _, row in df_depto.iterrows():
    folium.CircleMarker(
        location=[row["latitud"], row["longitud"]],  # columnas de df_poi
        radius=1,                   # lo más pequeño posible
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
    ).add_to(depto_layer)

# ======================
#  POIs (df_poi) - red
# ======================
for _, row in df_poi.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],  # columnas de df_poi
        radius=1,                   # lo más pequeño posible
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
    ).add_to(pois_layer)

# ==========================
#  Paraderos Bus (df_bus) - negro
# ==========================
for _, row in df_bus.iterrows():
    folium.CircleMarker(
        # ⚠️ Ajusta los nombres de columna si en df_bus no se llaman "latitude"/"longitude"
        location=[row["latitud"], row["longitud"]],
        radius=1,
        color="#000000",           # verde claro
        fill=True,
        fill_color="#000000",
        fill_opacity=0.6,
    ).add_to(bus_layer)

# ==========================
#  Estaciones Metro (df_metro) - naranjo
# ==========================
for _, row in df_metro.iterrows():
    folium.CircleMarker(
        # ⚠️ Ajusta los nombres si es necesario
        location=[row["latitud"], row["longitud"]],
        radius=1,
        color="#CCA700",
        fill=True,
        fill_color="#CCA700",
        fill_opacity=0.6,
    ).add_to(metro_layer)

# Agregar capas al mapa
depto_layer.add_to(mapa)
pois_layer.add_to(mapa)
bus_layer.add_to(mapa)
metro_layer.add_to(mapa)

# Control de capas para activar/desactivar cada tipo de punto
folium.LayerControl().add_to(mapa)

# Guardar mapa
mapa_path = "mapa_santiago_depto_poi_bus_metro.html"
mapa.save(mapa_path)
